<a href="https://colab.research.google.com/github/Vedant724/Rice-Quality-Project/blob/main/ricequality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib
import random

In [3]:
def generate_background_texture(size):
  texture = np.ones(size, dtype=np.uint8) * 230 # Light gray background
  cv2.randn(texture, 240, 10) # Add some Gaussian noise
  return texture


In [5]:
def create_rice_grain(center, axes):
# Create an empty mask with a transparent background
  rice_grain = np.zeros((axes[1] * 2, axes[0] * 2, 3), dtype=np.uint8)
  cv2.ellipse(rice_grain, (axes[0], axes[1]), (axes[0], axes[1] // 2), 0, 0, 360,
  (255, 255, 255), -1)
  noise = np.random.uniform(-5, 5, rice_grain.shape).astype(np.uint8)
  rice_grain = cv2.add(rice_grain, noise)
  return rice_grain

In [11]:
def create_synthetic_rice_images(num_images=100, image_size=(300, 300)):
  if not os.path.exists('rice_images'):
    os.makedirs('rice_images')
  data = []
  for i in range(num_images):
    image = generate_background_texture((image_size[0], image_size[1], 3))
    for _ in range(random.randint(30, 70)): # Random number of grains
# Random position and size for the grains
      center = (np.random.randint(20, image_size[1] - 20),
      np.random.randint(20, image_size[0] - 20))
      axes = (np.random.randint(15, 25), np.random.randint(7, 12)) # Wider
# Generate the rice grain
      rice_grain = create_rice_grain(center, axes)
# Calculate the position for placing the grain
      x_offset = center[0] - axes[0]
      y_offset = center[1] - axes[1] // 2
      if (0 <= x_offset < image_size[1]) and (0 <= y_offset < image_size[0]):
# Get the region of interest in the original image
        try:
          roi = image[y_offset:y_offset + rice_grain.shape[0], x_offset:x_offset
+ rice_grain.shape[1]]
# Create a mask from the rice grain
          mask = rice_grain != [0, 0, 0]
# Overlay the rice grain onto the original image only where it is no
          roi[mask] = rice_grain[mask]
        except Exception as e:
            print(f"Error placing rice grain: {e}")
      image = cv2.GaussianBlur(image, (3, 3), 0)
      filename = f'rice_images/rice_image_{i}.png'
# Save the image
      cv2.imwrite(filename, image)
# Assign a random price, for example, prices between $1 and $10
      price = np.random.uniform(1.0, 10.0)
# Append to the list
      data.append({'image_path': filename, 'price': price})
      df = pd.DataFrame(data)
      df.to_csv('rice_data.csv', index=False)
      print("Synthetic rice images created and saved as 'rice_data.csv'.")

In [12]:
def extract_features(image_path):
  image = cv2.imread(image_path)
  if image is None:
    return None
  image = cv2.resize(image, (300, 300)) # Match the size used in image
  mean_color = image.mean(axis=(0, 1))
  histogram = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0,
256])
  histogram = cv2.normalize(histogram, histogram).flatten()
  features = np.hstack((mean_color, histogram))
  return features


In [14]:
def display_images(num_images=10):
  for i in range(num_images):
    image_path = f'rice_images/rice_image_{i}.png'
    image = cv2.imread(image_path)
    if image is not None:
      cv2.imshow(f'Rice Image {i}', image)
      cv2.waitKey(0) # Wait for a key press to close the image
      cv2.destroyAllWindows() # Close the window after key press

In [16]:
def train_model():
# Load the synthetic dataset
  data = pd.read_csv('rice_data.csv')
  features = []
  prices = []
# Extract features and prices from the dataset
  for index, row in data.iterrows():
    image_path = row['image_path']
    price = row['price']
    feature = extract_features(image_path)
    if feature is not None:
      features.append(feature)
    X = np.array(features)
    y = np.array(prices)
# Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
# Train a model
    model = LinearRegression()
    model.fit(X_train, y_train)
# Save the model
    joblib.dump(model, 'rice_price_model.pkl')
    print("Model trained and saved as 'rice_price_model.pkl'.")

In [17]:
def predict_rice_price(image_path):
  model = joblib.load('rice_price_model.pkl')
  features = extract_features(image_path)
  if features is None:
    return None
  features = features.reshape(1, -1) # Reshape for the model
  predicted_price = model.predict(features)
  return predicted_price[0]

In [18]:
if __name__ == "__main__":
# Create synthetic rice-like images
  create_synthetic_rice_images(num_images=100, image_size=(300, 300))
# Display some images to visualize
  display_images(num_images=10) # Adjust the number if needed
# Train the model
  train_model()
# Predict price for a sample image
  image_path = 'rice_images/rice_image_0.png' # Change this if necessary
  price = predict_rice_price(image_path)
  print("Predicted Rice Price: $", price)

Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved as 'rice_data.csv'.
Synthetic rice images created and saved 

KeyboardInterrupt: 